**Introduction to PyQuil**


In [1]:
# installation
!pip install pyquil

all qubits are initialized to **|0>** state

In [2]:
import numpy as np
from pyquil.quil import Program
from pyquil.api import QVMConnection # for connecting to Rigetti Cloud

# importing the NOT gate (which is also the Pauli X gate) and Identity gate
from pyquil.gates import X, I

qvm = QVMConnection()

In [3]:
p = Program(I(0)) # apply identity gate on Qubit #0
print(p) # printing the underlying Quil Assembly code

I 0



In [4]:
wavefunction = qvm.wavefunction(p) # print the state vector
print(wavefunction)

(1+0j)|0>


In [14]:
# use classical control logic
p = Program()

for i in range(8):
    if i==3:
        p+=X(i)
    else:
        p += I(i)
    
wavefunction = qvm.wavefunction(p)
print(wavefunction)

(1+0j)|00001000>


In [7]:
# flipping some qubits

p = Program().inst(I(0),X(1)) # or equivalently Program(X(1))
wavefunction = qvm.wavefunction(p)

print(wavefunction)

(1+0j)|10>


In [17]:
# Applying CNOT gate
from pyquil.gates import CNOT
p = Program(CNOT(0,1))
w1 = qvm.wavefunction(p)
print('CNOT of |00> : ',w1)

q = Program(X(1),CNOT(1,0))
w2 = qvm.wavefunction(q)
print('CNOT of |10> : ',w2)

print('\nThe corresponding assembly code is')
print(q)

CNOT of |00> :  (1+0j)|00>
CNOT of |10> :  (1+0j)|11>

The corresponding assembly code is
X 1
CNOT 1 0



In [22]:
# import Hadamard gate
from pyquil.gates import H
# creating bell state

p = Program().inst(H(1),CNOT(1,0))
bell = qvm.wavefunction(p)

print(bell)

print('\nCorresponding assembly code\n',p)

(0.7071067812+0j)|00> + (0.7071067812+0j)|11>

Corresponding assembly code
 H 1
CNOT 1 0



In [37]:
# doing partial measurement on the bell state system on qubit 0 and place the result 
# in classical register indexed 3
from pyquil.gates import MEASURE

p = Program()
p.inst(
       H(0),
       CNOT(0, 1),
       MEASURE(0, 0),
       MEASURE(1, 1)
   )


print('Printing assembly code')
print(p)

trials = 10
print(qvm.run(p,[0,1],trials))




Printing assembly code
H 0
CNOT 0 1
MEASURE 0 [0]
MEASURE 1 [1]

[[1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [0, 0], [1, 1], [1, 1], [1, 1], [0, 0]]


In [42]:
# Quantum Teleportation
from pyquil.gates import Z

'''Entanglement - Preparation'''
p = Program()
p.inst(
    H(0),
    CNOT(0,1)
)
# qubit #0 is with A
# qubit #1 is with B

def EncodeQT(p,cbits):
    Gate = {
        '00':I(0),
        '01':X(0),
        '10':Z(0)
    }
    if cbits=='11':
        p.inst(Z(0),X(0))
    else:
        p.inst(Gate[cbits])

def DCodeQT(p):
    p.inst(
        CNOT(0,1),
        H(0),
        MEASURE(0, 0),
        MEASURE(1, 1)
    )
    
# on moon
EncodeQT(p,'11')

# far away on earth
DCodeQT(p)

print(qvm.run(p,[0,1]))
print('The Assembly Code')
print(p)

[[1, 1]]
The Assembly Code
H 0
CNOT 0 1
Z 0
X 0
CNOT 0 1
H 0
MEASURE 0 [0]
MEASURE 1 [1]

